In [1]:
import re
import pandas as pd

In [2]:
re_tags = re.compile('<.*?>') # re to remove all the tags
re_patent_number = re.compile('file="([^-]*)-') # re to extract patent number

re_paranthesis = re.compile('\(.*?\)') # removing stuff in paranthesis

In [3]:
number_of_patents = 0
number_of_design_patents = 0
patent_data = {"Patent_Number":[], "Description":[]};
description_flag = 0
description_string = 'x'
with open('ipg230207.xml', 'r') as f:
    for line in f:
        
#         if number_of_patents <= 1:
            #checking the number of patents
        if line == '<?xml version="1.0" encoding="UTF-8"?>\n':
            number_of_patents += 1

        #extract patent number
        if '<us-patent-grant lang="EN"' in line:
            patent_num = re_patent_number.findall(line)
            if 'USD0' in patent_num[0]:
                number_of_design_patents += 1

            patent_data["Patent_Number"].append(patent_num[0])

        if '<description id="description">' in line:
            description_string = ''
            description_flag = 1

        if line == '</description>\n':
            description_flag = 0
            patent_data["Description"].append(description_string)

        else:
            if description_flag == 1:
                line = re.sub(re_tags, '', line)
                description_string += line
#             elif description_string == '' or description_string == 'x':
#                 patent_data["Description"].append('')
                
                

In [4]:
print(number_of_patents)

6007


In [47]:
print(number_of_design_patents)

560


In [5]:
len(patent_data["Description"])

5882

In [4]:
# patent_data_og = patent_data
patent_data = patent_data_og

In [6]:
for idx, descr in enumerate(patent_data["Description"]):
#     if 'USD0' in patent_data["Patent_Number"][idx]:
#         patent_data["Description"][idx] = None
#     else:
    new_descr = ''
    new_descr_however = ''
    if 'however' in descr or 'However' in descr:
        flag_however = 0
        for line in descr.split('\n'):
            if len(line) > 20 and 'FIG' not in line and '&#x' not in line and 'THE' not in line and 'SUMMARY' in line:
                
                if ('however' in line or 'However' in line) and flag_however == 0:
                    flag_however = 1
                    new_descr = line
                    continue
                if flag_however < 3:
                    new_descr += ' ' + line
                    flag_however += 1
                if flag_however == 3:
                    flag_however = 0
    else:
        for line in descr.split('\n'):
            if len(line) > 20 and 'FIG' not in line and '&#x' not in line and 'THE' not in line and 'SUMMARY' in line:
                new_descr += ' ' + line

    if new_descr == '' or 'USD0' in patent_data["Patent_Number"][idx] or 'USPP0'in patent_data["Patent_Number"][idx]:
        patent_data["Description"][idx] = None
    else:
        new_descr = str(new_descr).strip()
        new_descr = re.sub(re_paranthesis, '', new_descr)
        patent_data["Description"][idx] = new_descr
#     print('lalala')


# skip USD - design patents - done
# evaluation of how well it describes - this week
# however, 3 paragraphs - 
# use examples while prompting - example - next week
# get 100 patents summrized - this week
# human problem - connection with the summary - last week?
# 

In [7]:
patent_data_df = pd.DataFrame.from_dict(patent_data)

In [73]:
patent_data_df.head()

,Patent_Number,Description
0,USD0977218,None
1,USD0977219,None
2,USD0977220,None
3,USD0977221,None
4,USD0977222,None


In [49]:
patent_data_df_og = patent_data_df

In [58]:
patent_data_df_og.head()

,Patent_Number,Description
0,USD0977218,None
1,USD0977219,None
2,USD0977220,None
3,USD0977221,None
4,USD0977222,None


In [77]:
import time
ts = str(time.time())
patent_data_df.to_csv("clean_data/patents_description_wahwah_"+ ts +".csv")

prompt: 'From the given description of a patent, can you tell what problem the patent is claiming to solve along with a 10 dimentional vector representation of your answer?' - or I can get embeddings from some other model as well and compare both?

In [50]:
patent_data_df['Description'].isna().sum()

570

In [12]:
# df = df[df['EPS'].notna()]

In [14]:
count_20 = 0
count_1500 = 0
patent_no_high_words = []
patent_no_list = []
num_of_words = 0
patent_data['length'] = []
for idx, line in enumerate(patent_data['Description']):
    if line:
        words = line.split(' ')
        patent_data['length'].append(len(words)) 
        if len(words) < 20:
#             print(len(words))
#             print(patent_data['Patent_Number'][idx])
            count_20 += 1
        elif len(words) > 1500:
            count_1500 += 1
            patent_no_high_words.append(patent_data['Patent_Number'][idx])
        else:
            num_of_words += len(words)
            patent_no_list.append(patent_data['Patent_Number'][idx])
        
print('the count is of patent descriptions with less than 20 words = '+ str(count_20))
print('the count is of patent descriptions with more than 1500 words = '+ str(count_1500))
print('total count of tokens from usable patents = '+ str(num_of_words))
print('num of patents that can be used now'len(patent_no_list))

the count is of patent descriptions with less than 20 words = 0
the count is of patent descriptions with more than 1500 words = 4196
total count of tokens from usable patents = 868463
1116


In [75]:
patent_data_df_og.describe()

,Patent_Number,Description
count,5882,5312
unique,5882,5230
top,USD0977218,"The description set forth herein, in connectio..."
freq,1,6


In [8]:
patent_data_df = patent_data_df.dropna()
patent_data_df.describe()

,Patent_Number,Description
count,5312,5312
unique,5312,5297
top,USRE049409,CROSS-REFERENCE TO RELATED APPLICATIONS/INCORP...
freq,1,3


In [13]:
patent_data_df.head()

,Patent_Number,Description
570,USRE049409,"In the present embodiment, the display unit 40..."
571,USRE049410,CROSS-REFERENCE TO RELATED APPLICATIONS NOTICE...
572,USRE049411,The locking catch 12 may be rectangular in sha...
573,USRE049412,BACKGROUND OF THE INVENTION 1. Field of the In...
574,USRE049413,Notice: More than one reissue application has ...


In [38]:
ts = str(time.time())
patent_data_df.to_csv("patents_description_filtered_"+ ts +".csv")

In [56]:
len(patent_no_list)

1931

In [15]:
#US11570970 - 572
patent_data_og['Description'][573]

'BACKGROUND OF THE INVENTION 1. Field of the Invention The present invention relates to method and apparatus for preparing and dispensing beverages from a pitcher and, more particularly, a pitcher for receiving the ingredients for mixing both alcoholic and non-alcoholic beverages where the ingredients for the beverage to be mixed and the amount of each ingredient are identified by a measuring ruler releasably attached to the pitcher where a plurality of measuring rulers are provided for mixing the ingredients of any number of beverages. 2. Description of the Prior Art It is a well known practice in the use of a mixing cup for cooking to identify on the cup the levels at which ingredients are added to the cup. For example, the exterior of a mixing cup is marked to identify the levels for filling ingredients from one-fourth cup to a full cup. Measure lines extend from the bottom of the cup to the open upper end of the cup. A one-half cup of sugar is measured by adding sugar to the cup to

In [18]:
string = ''
new_descr = ''
flag_however = 0
with open('USRE049411_descr.txt', 'r') as f:
    for text in f:
        text = re.sub(re_tags, '', text)
#         if line != '\n' and line != ' ':
        if 'however' in text or 'However' in text and flag_however == 0:
                flag_however = 1
                new_descr = text
                continue
        if flag_however < 3:
            new_descr += ' ' + text
            flag_however += 1
        if flag_however == 3:
            flag_however = 0
#         print('lalala')
    print(new_descr)
    

The locking catch 12 may be rectangular in shape, however it may also be an oblique surface as shown that slopes relative to the transverse direction in which the connection element 10 is inserted and that acts as a ramp for the sliding of the connection element 10.
 On its outer circumference, the body 3 of the female connector 2 has a shoulder 112 adjacent to each catch 12 and forming a stop for the connection element 10. In this example, the connection element 10 is in the form of two forks, a first fork of upside-down U-shape forming a lock 30, and a second fork of upside-down U-shape forming a yoke 14. The two forks are connected together via the bases of the U-shape.
 The first fork forming the lock 30 is designed to be placed behind the collar 7 of the male connector 6 so as to hold it and block it in the axial direction when the male connector 6 is fully pushed into the inside of the female connector 2. The lock 30 has two branches spaced apart from each other and extending in 